In [1]:
import os
os.environ["OPENAI_API_KEY"] = 'sk-yC52cYQkz0aMXJA5PySMUXdpVNBVNgKkdsv5kjEZpdx93PVs'
os.environ["OPENAI_BASE_URL"] = 'https://api.302.ai/v1/'


In [3]:
from langchain.prompts import PromptTemplate
prompt_template = """您是一位专业的鲜花店文案撰写员。
对于售价为 {price} 元的 {flower_name} ，您能提供一个吸引人的简短描述吗？
{format_instructions}"""

In [4]:
from langchain_openai import OpenAI
# 创建模型实例
model = OpenAI(model_name='gpt-3.5-turbo-instruct')

In [5]:
# 导入结构化输出解析器和ResponseSchema
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
# 定义我们想要接收的响应模式
response_schemas = [
    ResponseSchema(name="description", description="鲜花的描述文案"),
    ResponseSchema(name="reason", description="问什么要这样写这个文案")
]

In [6]:
# 创建输出解析器
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [11]:
output_parser

StructuredOutputParser(response_schemas=[ResponseSchema(name='description', description='鲜花的描述文案', type='string'), ResponseSchema(name='reason', description='问什么要这样写这个文案', type='string')])

In [7]:
# 获取格式指示
format_instructions = output_parser.get_format_instructions()

In [8]:
# 根据模板创建提示，同时在提示中加入输出解析器的说明
prompt = PromptTemplate.from_template(prompt_template, 
                partial_variables={"format_instructions": format_instructions}) 

In [12]:
# 数据准备
flowers = ["玫瑰", "百合", "康乃馨"]
prices = ["50", "30", "20"]

# 创建一个空的DataFrame用于存储结果
import pandas as pd

df = pd.DataFrame(columns=["flower", "price", "description", "reason"]) # 先声明列名

for flower, price in zip(flowers, prices):
    # 根据提示准备模型的输入
    input = prompt.format(flower_name=flower, price=price)
    print(input)
    # 获取模型的输出
    output = model.invoke(input)
    
    # 解析模型的输出（这是一个字典结构）
    parsed_output = output_parser.parse(output)

    # 在解析后的输出中添加“flower”和“price”
    parsed_output['flower'] = flower
    parsed_output['price'] = price

    # 将解析后的输出添加到DataFrame中
    df.loc[len(df)] = parsed_output  

# 打印字典 dataframe 转字典 
# 其中每个字典代表 DataFrame 中的一行 
# DataFrame 是一种二维表格型数据结构
# 而字典（dict）是键值对的集合
print(df.to_dict(orient='records'))

您是一位专业的鲜花店文案撰写员。
对于售价为 50 元的 玫瑰 ，您能提供一个吸引人的简短描述吗？
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"description": string  // 鲜花的描述文案
	"reason": string  // 问什么要这样写这个文案
}
```
您是一位专业的鲜花店文案撰写员。
对于售价为 30 元的 百合 ，您能提供一个吸引人的简短描述吗？
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"description": string  // 鲜花的描述文案
	"reason": string  // 问什么要这样写这个文案
}
```
您是一位专业的鲜花店文案撰写员。
对于售价为 20 元的 康乃馨 ，您能提供一个吸引人的简短描述吗？
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"description": string  // 鲜花的描述文案
	"reason": string  // 问什么要这样写这个文案
}
```
[{'flower': '玫瑰', 'price': '50', 'description': '这束美丽的玫瑰代表着爱情的纯洁和美好，是表达爱意的最佳选择。', 'reason': '这样的文案能够吸引顾客的注意，让他们感受到浪漫和温馨，从而激发购买欲望。玫瑰作为最受欢迎的鲜花之一，能够让顾客感受到自己的爱情和感情被重视，从而愿意花费50元购买。'}, {'fl

In [10]:
# 保存DataFrame到CSV文件
df.to_csv("flowers_with_descriptions.csv", index=False)